# 0.0 Importações e Bibliotecas

In [1]:
# !pip install inflection
# !pip install xgboost
# !pip install lightgbm
# !pip install category_encoders

In [2]:
import pandas as pd
import inflection

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn import model_selection as ms
from sklearn import metrics

from xgboost import XGBClassifier
from sklearn import ensemble as en
from lightgbm import LGBMClassifier
from category_encoders import target_encoder

# 0.1 Carregamento dos Dados

In [3]:
train_raw = pd.read_csv('/work/train.csv')
test_raw = pd.read_csv('/work/test.csv')

In [4]:
train_raw = train_raw.dropna()

X = train_raw.drop('Reserva Cancelada', axis=1)
y = train_raw['Reserva Cancelada'].copy()

# Split data into train and test
x_train, x_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.20, random_state=42)

df1 = pd.concat([x_train, y_train], axis=1)

# 1.0 Descrição dos dados

In [5]:
df1.head()

,id,Classificação do hotel,Meses da reserva até o check-in,Número de pernoites reservadas,Número de hospedes,Regime de alimentação,Nacionalidade,Forma de Reserva,Já se hospedou anterioremente,Tipo do quarto reservado,Reserva feita por agência de turismo,Reserva feita por empresa,Reserva com Estacionamento,Reserva com Observações,Reserva Cancelada
22283,73581,4 estrelas,6,6,1.0,Café da manha,France,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
36654,72263,4 estrelas,7,3,1.0,Sem refeicao,Switzerland,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,1
6398,91396,4 estrelas,4,4,2.0,Café da manha,Germany,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0
54899,50037,4 estrelas,1,1,1.0,Café da manha,Italy,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
37656,44151,4 estrelas,3,5,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1


In [6]:
df1.shape

(56850, 15)

In [7]:
old_cols = df1.columns
snakecase = lambda x : inflection.parameterize(x, separator='_')

cols = list(map(snakecase, old_cols))

df1.columns = cols


old_cols = x_test.columns
snakecase = lambda x : inflection.parameterize(x, separator='_')

cols = list(map(snakecase, old_cols))

x_test.columns = cols

In [8]:
df1.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check-in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
dtype: object

In [9]:
df1.isna().sum()

id                                      0
classificacao_do_hotel                  0
meses_da_reserva_ate_o_check-in         0
numero_de_pernoites_reservadas          0
numero_de_hospedes                      0
regime_de_alimentacao                   0
nacionalidade                           0
forma_de_reserva                        0
ja_se_hospedou_anterioremente           0
tipo_do_quarto_reservado                0
reserva_feita_por_agencia_de_turismo    0
reserva_feita_por_empresa               0
reserva_com_estacionamento              0
reserva_com_observacoes                 0
reserva_cancelada                       0
dtype: int64

In [10]:
# df1 = df1.dropna()

# x_test = x_test.dropna()

In [11]:
df1.describe()

,id,meses_da_reserva_ate_o_check-in,numero_de_pernoites_reservadas,numero_de_hospedes,reserva_cancelada
count,56850.000000,56850.000000,56850.000000,56850.000000,56850.000000
mean,60424.381548,7.080967,15.930413,14.383624,0.374389
std,34657.114569,25.530301,98.658305,96.962955,0.483969
min,1.000000,1.000000,0.000000,0.000000,0.000000
25%,30577.250000,1.000000,2.000000,2.000000,0.000000
50%,60595.500000,3.000000,3.000000,2.000000,0.000000
75%,90566.500000,6.000000,4.000000,2.000000,1.000000
max,120263.000000,311.000000,1172.000000,1153.000000,1.000000


In [12]:
df1[df1['meses_da_reserva_ate_o_check-in'] == 311.0]

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check-in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada
14977,1189,5 estrelas,311,5,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,1
64944,77801,4 estrelas,311,2,2.0,Café da manha,Spain,B2B,Não,Amethyst,Sim,Não,Não,Nenhuma,1
71016,99752,4 estrelas,311,3,2.0,Café da manha,Spain,Agência,Não,Pink Sapphire,Sim,Não,Não,Nenhuma,0
43684,112144,4 estrelas,311,3,2.0,Café da manha e jantar,Belgium,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0
5387,37620,5 estrelas,311,7,1.0,Café da manha,United Kingdom,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0


# 2.0 Feature Engineering

In [13]:
df2 = df1.copy()

## 2.1 Hipóteses

1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 
10. 

## 2.2 Feature Engineering

# 3.0 Filtragem dos Dados

In [14]:
df3 = df2.copy()

## 3.1 Filtragem das Linhas

## 3.2 Filtragem das Colunas

# 4.0 Análise Exploratória dos Dados (AED)

In [15]:
df4 = df3.copy()

## 4.1.1 Análise Univariada

In [16]:
df4[df4['reserva_cancelada'] == 1]

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check-in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada
22283,73581,4 estrelas,6,6,1.0,Café da manha,France,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
36654,72263,4 estrelas,7,3,1.0,Sem refeicao,Switzerland,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,1
54899,50037,4 estrelas,1,1,1.0,Café da manha,Italy,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
37656,44151,4 estrelas,3,5,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
10226,60836,4 estrelas,3,4,2.0,Café da manha,France,Balcão,Não,Amethyst,Sim,Não,Não,1 a 3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69023,52789,4 estrelas,7,3,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
65925,72119,4 estrelas,5,4,2.0,Sem refeicao,United States,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
16237,74981,4 estrelas,10,2,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
55726,69220,4 estrelas,4,3,1.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1


## 4.1.2 Variáveis Numéricas

## 4.1.3 Variáveis Categóricas

## 4.1.4 Variável resposta

## 4.2.1 Análise Multivariada

## 4.2.1 Atributos Numéricos

## 4.2.2 Atributos Categóricos

# 5.0 Preparação dos dados

In [17]:
df5 = df4.copy()

## 5.1 Normalização

## 5.2 Reescalonar (Rescaling)

# 5.3 Transformação

## 5.3.1 Encoding

In [18]:
df5.columns

Index(['id', 'classificacao_do_hotel', 'meses_da_reserva_ate_o_check-in',
       'numero_de_pernoites_reservadas', 'numero_de_hospedes',
       'regime_de_alimentacao', 'nacionalidade', 'forma_de_reserva',
       'ja_se_hospedou_anterioremente', 'tipo_do_quarto_reservado',
       'reserva_feita_por_agencia_de_turismo', 'reserva_feita_por_empresa',
       'reserva_com_estacionamento', 'reserva_com_observacoes',
       'reserva_cancelada'],
      dtype='object')

In [19]:
df5 = df2.copy()

In [20]:
df5.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check-in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
dtype: object

In [21]:
old_cols = test_raw.columns
snakecase = lambda x : inflection.parameterize(x, separator='_')

cols = list(map(snakecase, old_cols))

test_raw.columns = cols


le = LabelEncoder()
te = target_encoder.TargetEncoder(handle_unknown='value')

df5['regime_de_alimentacao'] = le.fit_transform(df5['regime_de_alimentacao'])
x_test['regime_de_alimentacao'] = le.transform(x_test['regime_de_alimentacao'])
test_raw['regime_de_alimentacao'] = le.transform(test_raw['regime_de_alimentacao'])

df5['classificacao_do_hotel'] = le.fit_transform(df5['classificacao_do_hotel'])
x_test['classificacao_do_hotel'] = le.transform(x_test['classificacao_do_hotel'])
test_raw['classificacao_do_hotel'] = le.transform(test_raw['classificacao_do_hotel'])

df5['nacionalidade'] = te.fit_transform(df5['nacionalidade'], df5['reserva_cancelada'])
x_test['nacionalidade'] = te.transform(x_test['nacionalidade'])
test_raw['nacionalidade'] = te.transform(test_raw['nacionalidade'])

df5['forma_de_reserva'] = le.fit_transform(df5['forma_de_reserva'])
x_test['forma_de_reserva'] = le.transform(x_test['forma_de_reserva'])
test_raw['forma_de_reserva'] = le.transform(test_raw['forma_de_reserva'])

df5['ja_se_hospedou_anterioremente'] = le.fit_transform(df5['ja_se_hospedou_anterioremente'])
x_test['ja_se_hospedou_anterioremente'] = le.transform(x_test['ja_se_hospedou_anterioremente'])
test_raw['ja_se_hospedou_anterioremente'] = le.transform(test_raw['ja_se_hospedou_anterioremente'])

df5['tipo_do_quarto_reservado'] = le.fit_transform(df5['tipo_do_quarto_reservado'])
x_test['tipo_do_quarto_reservado'] = le.transform(x_test['tipo_do_quarto_reservado'])
test_raw['tipo_do_quarto_reservado'] = le.transform(test_raw['tipo_do_quarto_reservado'])

df5['reserva_feita_por_agencia_de_turismo'] = le.fit_transform(df5['reserva_feita_por_agencia_de_turismo'])
x_test['reserva_feita_por_agencia_de_turismo'] = le.transform(x_test['reserva_feita_por_agencia_de_turismo'])
test_raw['reserva_feita_por_agencia_de_turismo'] = le.transform(test_raw['reserva_feita_por_agencia_de_turismo'])

df5['reserva_feita_por_empresa'] = le.fit_transform(df5['reserva_feita_por_empresa'])
x_test['reserva_feita_por_empresa'] = le.transform(x_test['reserva_feita_por_empresa'])
test_raw['reserva_feita_por_empresa'] = le.transform(test_raw['reserva_feita_por_empresa'])

df5['reserva_com_estacionamento'] = le.fit_transform(df5['reserva_com_estacionamento'])
x_test['reserva_com_estacionamento'] = le.transform(x_test['reserva_com_estacionamento'])
test_raw['reserva_com_estacionamento'] = le.transform(test_raw['reserva_com_estacionamento'])

df5['reserva_com_observacoes'] = le.fit_transform(df5['reserva_com_observacoes'])
x_test['reserva_com_observacoes'] = le.transform(x_test['reserva_com_observacoes'])
test_raw['reserva_com_observacoes'] = le.transform(test_raw['reserva_com_observacoes'])


# 6.0 Seleção dos Atributos

In [22]:
df6 = df5.copy()

## 6.1 Separação dos Dados para Treino e Teste

## 6.1 Escolha das Colunas / Atributos

# 7.0 Modelagem de Machine Learning

In [23]:
df7 = df5.copy()

# from xgboost import XGBClassifier
# from sklearn import ensemble as en
# from lightgbm import LGBMClassifier

In [24]:
X = df7.drop('reserva_cancelada', axis=1)
y = df7['reserva_cancelada'].copy()

## 7.1 Modelo Random Forest Classifier

In [25]:
rf_model = en.RandomForestClassifier()

rf_model.fit(X, y)

RandomForestClassifier()

In [26]:
y_hat = rf_model.predict(x_test)

In [27]:
metrics.f1_score(y_test, y_hat)

0.962406015037594

## 7.2 Modelo x

## 7.3 Modelo x

## 7.4 Modelo x

## 7.5 Modelo x

## 7.6 Avaliação dos Modelos

# 8.0 Ajuste do Modelo (Hyperparameter Fine Tuning)

In [28]:
df8 = df7.copy()

## 8.1 Busca Aleatória (Random Search)

## 8.2 Modelo Final

# 9.0 Submissão

In [29]:
df9 = df8.copy()

In [37]:
test_raw['numero_de_hospedes'] = test_raw['numero_de_hospedes'].fillna(df1['numero_de_hospedes'].median())

In [38]:
test_raw.isna().sum()

id                                      0
classificacao_do_hotel                  0
meses_da_reserva_ate_o_check-in         0
numero_de_pernoites_reservadas          0
numero_de_hospedes                      0
regime_de_alimentacao                   0
nacionalidade                           0
forma_de_reserva                        0
ja_se_hospedou_anterioremente           0
tipo_do_quarto_reservado                0
reserva_feita_por_agencia_de_turismo    0
reserva_feita_por_empresa               0
reserva_com_estacionamento              0
reserva_com_observacoes                 0
dtype: int64

In [40]:
y_hat = rf_model.predict(test_raw)

In [48]:
pd.Series(y_hat, name='Reserva Cancelada')

0        0
1        1
2        0
3        1
4        0
        ..
48101    0
48102    0
48103    0
48104    1
48105    1
Name: Reserva Cancelada, Length: 48106, dtype: int64

In [54]:
test = pd.concat([test_raw, pd.Series(y_hat)], axis=1).rename(columns={0: 'Reserva Cancelada'})

test[['id', 'Reserva Cancelada']].to_csv('/work/submit1.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1f2e6561-5818-46b0-8797-0a88d12bf58e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>